In [163]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.col

val ldata = spark.read.format("csv").option("header","true").load("/Users/pulkit/data/london_crime_by_lsoa.csv")

In [9]:
ldata.printSchema

root
 |-- lsoa_code: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- major_category: string (nullable = true)
 |-- minor_category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)



In [10]:
ldata.count

13490604

In [17]:
ldata.limit(5).show

+---------+----------+--------------------+--------------------+-----+----+-----+
|lsoa_code|   borough|      major_category|      minor_category|value|year|month|
+---------+----------+--------------------+--------------------+-----+----+-----+
|E01001116|   Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|E01001646| Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|E01000677|   Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|E01003774| Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|E01004563|Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
+---------+----------+--------------------+--------------------+-----+----+-----+



In [29]:
val ldatana = ldata.drop("lsoa_code")
                    .na.drop("all")
                    .na.fill("Unknown",Seq("major_category"))

In [64]:
val total_borough = ldatana.select("borough").distinct().show(3)

+----------+
|   borough|
+----------+
|   Croydon|
|Wandsworth|
|    Bexley|
+----------+
only showing top 3 rows



In [58]:
val years = List("2015", "2016")
val hackney_data = ldatana.where( col("borough") === "Hackney" || col("year").isin( years: _*))

In [60]:
hackney_data.sample(fraction=0.1).show

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|             Lambeth|Violence Against ...|      Other violence|    0|2015|    4|
|           Southwark|               Drugs| Possession Of Drugs|    0|2015|    3|
|              Newham|            Burglary|Burglary in Other...|    0|2015|    2|
|Kingston upon Thames|  Theft and Handling|Motor Vehicle Int...|    0|2016|    5|
|           Islington|     Sexual Offences|        Other Sexual|    0|2016|   10|
|Hammersmith and F...|               Drugs| Possession Of Drugs|    1|2016|    2|
|             Enfield|  Theft and Handling|Motor Vehicle Int...|    1|2015|    7|
|             Bromley|     Criminal Damage|Criminal Damage T...|    0|2015|    9|
|               Brent|Other Notifiable ...|    Other Notifiable|    0|2016|    4|
|             Ha

In [105]:
val boroughCrimeCount = ldatana.groupBy("borough").count()

In [106]:
boroughCrimeCount.show(3)

+----------+------+
|   borough| count|
+----------+------+
|   Croydon|602100|
|Wandsworth|498636|
|    Bexley|385668|
+----------+------+
only showing top 3 rows



In [143]:
val boroughconvc = ldatana.groupBy("borough").agg("value" -> "sum").withColumnRenamed("sum(value)","convictions")

In [147]:
val tbc = boroughconvc.agg(sum("convictions"))
tbc.schema

StructType(StructField(sum(convictions),DoubleType,true))

In [119]:
val totalConvictions = tbc.collect()(0)(0)

In [129]:
val boroughPercentContribution = boroughconvc.
                                withColumn("contribution",round(col("convictions") / totalConvictions * 100,2))
                                

In [123]:
boroughPercentContribution.printSchema

root
 |-- borough: string (nullable = true)
 |-- convictions: double (nullable = true)
 |-- contribution: double (nullable = true)



In [139]:
boroughPercentContribution.orderBy(col("convictions").desc).show(10)

+-------------+-----------+------------+
|      borough|convictions|contribution|
+-------------+-----------+------------+
|  Westminster|   455028.0|        7.06|
|      Lambeth|   292178.0|        4.53|
|    Southwark|   278809.0|        4.32|
|       Camden|   275147.0|        4.27|
|       Newham|   262024.0|        4.06|
|      Croydon|   260294.0|        4.04|
|       Ealing|   251562.0|         3.9|
|    Islington|   230286.0|        3.57|
|Tower Hamlets|   228613.0|        3.55|
|        Brent|   227551.0|        3.53|
+-------------+-----------+------------+
only showing top 10 rows



In [158]:
val convictionsMonthly = ldatana.where(col("year") === "2014").
                                groupBy("month").
                                agg(sum("value")).
                                withColumnRenamed("sum(value)","convictions")

In [159]:
val totalConvictionsMonthly = convictionsMonthly.agg(sum("convictions")).collect()(0)(0)

In [160]:
val toatalPercentContributionMon = convictionsMonthly.
                                withColumn("percent",round(col("convictions") / totalConvictions * 100,2))

In [161]:
toatalPercentContributionMon.columns

Array(month, convictions, percent)

In [166]:
toatalPercentContributionMon.orderBy(col("percent").desc).show

+-----+-----------+-------+
|month|convictions|percent|
+-----+-----------+-------+
|   10|    60537.0|   0.94|
|   11|    59704.0|   0.93|
|    7|    58564.0|   0.91|
|   12|    57565.0|   0.89|
|    3|    57669.0|   0.89|
|    6|    57039.0|   0.88|
|    9|    56933.0|   0.88|
|    5|    56327.0|   0.87|
|    1|    55515.0|   0.86|
|    8|    55641.0|   0.86|
|    4|    53467.0|   0.83|
|    2|    51222.0|   0.79|
+-----+-----------+-------+



In [176]:
ldatana.select("year").agg(max("year"), min("year")).show()

+---------+---------+
|max(year)|min(year)|
+---------+---------+
|     2016|     2008|
+---------+---------+



In [188]:
ldatana.describe("year","value").show

+-------+-----------------+------------------+
|summary|             year|             value|
+-------+-----------------+------------------+
|  count|         13490604|          13490604|
|   mean|           2012.0|0.4779443529733732|
| stddev|2.581988993167432|1.7715128643049873|
|    min|             2008|                 0|
|    max|             2016|                99|
+-------+-----------------+------------------+

